In [36]:
from fileinput import filename
import pandas as pd
import os
import glob
import re


# Lista de palavras ignoradas (já definida)
palavras_ignoradas = {
    "NETO", "NEGRI", "NOVE", "N KM", "Niemeyer", "NICOLAU", "NEREU", "NATAL", "NELSINA", "NILO", "Andrade", 
    "N KM430", "SALGADO", "SALMÃO", "NOVICKI", "Nébias", "NÚMERO", "NEVES", "NOTARI", "NEGRO", "NAGANO", 
    "ANDRÉ", "NERY", "NOBREGA", "NAKAZATO", "N BRCAO", "NEBLINA", "NOVEMBRO", "SALVESTRINI", "APOIO", 
    "SALLES", "NOGUEIRA", "NY", "NASCIMENTO", "NACOES", "NADER", "NICANOR", "Andorra", "NAZEAZENO", 
    "NORTE", "NOBORU", "NE", "SALES", "SALLA", "SALLE", "NOVELINO", "NELSON", "NS", "NAZARÉ", "NUNES", 
    "ANDRE", "N ETN", "NAGASHIMA", "NOSSA", "NAZARENO", "NORDESTINA", "NOBRE", "NATANAEL", "SALUSTIANO", 
    "ANDRADINA", "ANDRADAS", "NONOAI", "NEGRAS", "Noticias", "NELSIA", "N CH", "NATINGUI", "NORONHA", 
    "NU1700", "NSA", "BLUMENAU", "N GLEBA", "NAVES", "ANDREA", "SALVADOR", "N KM6", "NH", "NOVA", 
    "SALTINHO", "NICOLA", "NAGIB", "N GALPAO", "ANDREAZZA", "NASCENTE", "NONATO", "NAKATA", "NAÇOES", 
    "N DIBBI", "NA APULCRO", "APRIGIO", "ANDARAI", "NOVAES", "NS APARECIDA", "NOVIS", "NANUQUE", 
    "NARCISO", "NABUCO", "NCT", "SALVATORE", "NAZARETH", "ANDRELINA", "ANDRADE", "ANDORRA", "NÉBIAS",
    "NIEMEYER", "NOTÍCIAS", "NOTICIAS"
}

def extrair_endereco(lessee_street):
    # Remover as palavras ignoradas
    palavras = lessee_street.split()
    palavras = [palavra for palavra in palavras if palavra.upper() not in palavras_ignoradas]
    
    # Reconstruir o endereço sem as palavras ignoradas
    endereco_limpo = " ".join(palavras)

    # Expressões regulares para detectar e separar os componentes
    # Logradouro: Pode ser "AV", "R", "EST", "ROD", etc.
    logradouro_regex = r"^(AV|R|EST|ROD|DT|QUADRA|UNIDADE|SÃO|SANTA|VIA|PRAÇA|AL|TRAVESSA|RUA|ALAMEDA|PARQUE|LARGO)[^\d]+"
    numero_complemento_regex = r"(\d+|S/N)(.*)"

    # Encontrar o logradouro
    logradouro_match = re.match(logradouro_regex, endereco_limpo)
    logradouro = logradouro_match.group(0).strip() if logradouro_match else ''

    # Encontrar o número e complemento
    numero_complemento_match = re.search(numero_complemento_regex, endereco_limpo)
    if numero_complemento_match:
        numero = numero_complemento_match.group(1).strip()
        complemento = numero_complemento_match.group(2).strip()
    else:
        numero = ''
        complemento = ''

    # Ajustes para casos específicos
    # Remover vírgulas extra no complemento
    complemento = complemento.replace(",", "")
    
    # Casos específicos de complementos com mais de uma parte
    if "ANDAR" in complemento or "SL" in complemento or "CONJ" in complemento:
        complemento = complemento.replace("  ", " ").strip()

    return logradouro, numero, complemento

path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\'
file_name = 'PMD-CMS.xlsx'
full_file_name = path + file_name
sheet_name = 'after synch'

df_master = pd.read_excel(full_file_name, sheet_name=sheet_name)

HEADERS = df_master.iloc[0, :].tolist()

df_master.columns = HEADERS
df_master = df_master.iloc[1:,0:13]


df_master[['Numero', 'Complemento', 'LesseeStreet']] = df_master['LesseeStreet'].apply(lambda x: pd.Series(extract_number_and_complement(x)))

COLUMNS = [
    'CompanyNumber', 'Branch', 
    'ContractNumber', 'Bearbeitungsstand', 
    'Rueckstand', 'LesseeStreet','Numero', 
    'Complemento','LesseePostCode', 'LesseeCity',
    'LesseCountry', 'LesseeCounty',
    'LesseeState', 'LesseeName']

df_master = df_master.loc[:,COLUMNS]

DE_PARA_CEPS = pd.read_excel(full_file_name, sheet_name='CEPS')

df_master = df_master.merge(DE_PARA_CEPS,left_on='LesseePostCode',right_on='CEP_OLD',how='left')

COLUMNS = [
    'CompanyNumber', 'Branch', 
    'ContractNumber', 'Bearbeitungsstand', 
    'Rueckstand', 'NewLesseeStreet', 'LesseeStreet','CEP_NEW','Numero', 
    'Complemento','LesseePostCode', 'LesseeCity',
    'LesseCountry', 'LesseeCounty',
    'LesseeState', 'LesseeName']

df_master["NewLesseeStreet"] = df_master["LesseeStreet"]
df_master = df_master.loc[ : , COLUMNS ]

path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\finalizados\\'
file_name = 'base_cep_consolidada.xlsx'

full_file_name = path + file_name
BASE_CEP_API = pd.read_excel(full_file_name)

COLUMNS = [
    'cep_validado', 'logradouro',
    'bairro', 'cidade', 'uf', 'Estado'
]

BASE_CEP_API = BASE_CEP_API.loc[ : , COLUMNS ]

df_master = df_master.merge(BASE_CEP_API, how='left', left_on='CEP_NEW', right_on='cep_validado')

COLUMNS = [
    'CompanyNumber', 'LesseeName','Branch',
    'ContractNumber', 'Bearbeitungsstand',
    'Rueckstand', 'CEP_NEW', 'NewLesseeStreet','logradouro',
    'Numero', 'Complemento', 'bairro', 'cidade',
    'uf', 'Estado'
]
df_master = df_master.loc[ : , COLUMNS ]


path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\finalizados\\'
file_name = 'enderecos_processados.xlsx'
full_file_name = path + file_name

df_master.to_excel(full_file_name, index=False)
